#  Setup stuff: Connectivity

In [1]:

#  Setting display options

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


--


In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, EybCTDvSSMabzBiTNduP3BpK7nENdCZ1CrUQyRFzew5w, 0>


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [6]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

# df_airports.head(20)
print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+----------------+----------------+---------+
|    | airport_code   | airport_name   | LABEL   |
|----+----------------+----------------+---------|
|  0 | MKE            | Milwaukee      | Airport |
|  1 | ORD            | Chicago O-Hare | Airport |
|  2 | SJC            | San Jose       | Airport |
|  3 | DEN            | Denver         | Airport |
+----+----------------+----------------+---------+


In [7]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

# df_flights.head(30)
print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [8]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

--


In [9]:

#  This works
#
l_result = my_graph.query_unpaginated("""

   MATCH ( n ) 
   RETURN COUNT(*)
   
   """)

display(l_result)


#  /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't
#  allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
#    self._response = response



          0/? [?op/s]

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


,COUNT(*)
0,4


In [10]:

#  This fails
#
l_result = my_graph.query_unpaginated("""

   MATCH ( n ) 
   RETURN COUNT()
   
   """)

display(l_result)


#  OperationError: ATkc2xoW5Kf1NV7ayUA2w2Lg2R488ez6ieiAMZkk7Z3D-28TVyx6Fuzr8cQ6Dw backtrace (QueryOperation.cpp:66):
#     backtrace (Network.h:331): backtrace (QueryOperation.cpp:69): backtrace (QueryClient.cpp:655): opgraph check
#     failed (OpGraph.cpp:494): Type Checking Pass (OpGraphErrorChecker.cpp:12): backtrace (TypeChecker.cpp:1763):
#     backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:729):
#     backtrace (TypeChecker.cpp:1163): backtrace (TypeChecker.cpp:1685): checking func count (TypeChecker.cpp:288):
#     backtrace (TypeChecker.cpp:947): expect 1 argument to count, got 0 (CountFunc.cpp:85): TCK = SyntaxError:InvalidNumberOfArguments
#  Katana = ArgumentError:InvalidNumberOfArguments: TCK = SyntaxError:InvalidNumberOfArguments
#  Katana = ArgumentError:InvalidNumberOfArguments: TCK = SyntaxError:InvalidNumberOfArguments
#  Katana = ArgumentError:InvalidNumberOfArguments



          0/? [?op/s]

OperationError: GJesa56XtuVVycZApftKrHyWkpZABNtNcRw2Xr9Qm8df-2ehFdnhhLQqyEL272 backtrace (QueryOperation.cpp:66): backtrace (Network.h:331): backtrace (QueryOperation.cpp:69): backtrace (QueryClient.cpp:655): opgraph check failed (OpGraph.cpp:494): Type Checking Pass (OpGraphErrorChecker.cpp:12): backtrace (TypeChecker.cpp:1763): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:753): backtrace (OpGraph.cpp:729): backtrace (TypeChecker.cpp:1163): backtrace (TypeChecker.cpp:1685): checking func count (TypeChecker.cpp:288): backtrace (TypeChecker.cpp:947): expect 1 argument to count, got 0 (CountFunc.cpp:85): TCK = SyntaxError:InvalidNumberOfArguments
Katana = ArgumentError:InvalidNumberOfArguments: TCK = SyntaxError:InvalidNumberOfArguments
Katana = ArgumentError:InvalidNumberOfArguments: TCK = SyntaxError:InvalidNumberOfArguments
Katana = ArgumentError:InvalidNumberOfArguments